### The files for this code are not on the github repo, please reach out to the authors if you would like to verify or compute the stats in this code. 

In [2]:
import sys
import os, glob
import pdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import minimize
import math
import pingouin as pg

In [3]:
def strip_new_line_char(string):
    ""
    if "\n" in string:
        return string[:-1]
    return string
           
def bestFitParametersProbabilistic(folder,subject,sensoryStd):
    subjectArray = np.zeros((100,4))
    for iterations in range(1,101):
        file_dev = folder+'results_'+subject+'_'+str(iterations)+'.txt'
        with open(file_dev, 'r') as f:
            filelines = f.readlines()
        fileArray = np.zeros((1,4))
        iline = 0
        while iline < len(filelines):
            if filelines[iline].startswith('[') and filelines[iline].endswith(']\n'):
                lineWithNewLineChar = strip_new_line_char(filelines[iline])
                lineElements = lineWithNewLineChar.split(",")
                temp = np.zeros((1,4))
                temp[:,3] = float(lineElements[3][:-1])
                temp[:,2] = float(lineElements[2][:-2])
                temp[:,1] = float(lineElements[1][:])
                temp[:,0] = float(lineElements[0][8:])
                fileArray = np.append(fileArray,temp,axis=0)
                iline += 1
            else:
                line1WithNewLineChar = strip_new_line_char(filelines[iline])
                lineElementsOfFirstLine = line1WithNewLineChar.split(",")
                line2WithNewLineChar = strip_new_line_char(filelines[iline+1])
                lineElementsOfSecondLine = line2WithNewLineChar.split(",")
                lineElements = lineElementsOfFirstLine[:-1] + lineElementsOfSecondLine
                temp = np.zeros((1,4))
                temp[:,3] = float(lineElements[3][:-1])
                temp[:,2] = float(lineElements[2][:-2])
                temp[:,1] = float(lineElements[1][:])
                temp[:,0] = float(lineElements[0][8:])
                fileArray = np.append(fileArray,temp,axis=0)
                iline += 2 

        fileArraySSConstrained_upperlimit = fileArray[fileArray[:,0]<sensoryStd+0.02]
        fileArraySSConstrained_lowerlimit = fileArraySSConstrained_upperlimit[fileArraySSConstrained_upperlimit[:,0]>sensoryStd-0.02]
        subjectArray[iterations-1,:] = fileArraySSConstrained_lowerlimit[np.argmin(fileArraySSConstrained_lowerlimit[:,-1])]
    return subjectArray

def bestFitParametersSignal(folder,subject,sensoryStd):
    subjectArray = np.zeros((100,3))
    for iterations in range(1,101):
        file_dev = folder+'results_'+subject+'_'+str(iterations)+'.txt'
        with open(file_dev, 'r') as f:
            filelines = f.readlines()
        fileArray = np.zeros((1,3))
        iline = 0
        while iline < len(filelines):
            if filelines[iline].startswith('[') and filelines[iline].endswith(']\n'):
                lineWithNewLineChar = strip_new_line_char(filelines[iline])
                lineElements = lineWithNewLineChar.split(",")
                temp = np.zeros((1,3))
                temp[:,2] = float(lineElements[2][:-1])
                temp[:,1] = float(lineElements[1][:-2])
                temp[:,0] = float(lineElements[0][8:])
                fileArray = np.append(fileArray,temp,axis=0)
                iline += 1
            else:
                line1WithNewLineChar = strip_new_line_char(filelines[iline])
                lineElementsOfFirstLine = line1WithNewLineChar.split(",")
                line2WithNewLineChar = strip_new_line_char(filelines[iline+1])
                lineElementsOfSecondLine = line2WithNewLineChar.split(",")
                lineElements = lineElementsOfFirstLine[:-1] + lineElementsOfSecondLine
                temp = np.zeros((1,3))
                temp[:,2] = float(lineElements[2][:-1])
                temp[:,1] = float(lineElements[1][:-2])
                temp[:,0] = float(lineElements[0][8:])
                fileArray = np.append(fileArray,temp,axis=0)
                iline += 2 

        fileArraySSConstrained_upperlimit = fileArray[fileArray[:,0]<sensoryStd+0.02]
        fileArraySSConstrained_lowerlimit = fileArraySSConstrained_upperlimit[fileArraySSConstrained_upperlimit[:,0]>sensoryStd-0.02]
        subjectArray[iterations-1,:] = fileArraySSConstrained_lowerlimit[np.argmin(fileArraySSConstrained_lowerlimit[:,-1])]
    return subjectArray

def bestFitParametersRandom(folder,subject):
    subjectArray = np.zeros((100,3))
    for iterations in range(1,101):
        file_dev = folder+'results_'+subject+'.txt'
        with open(file_dev, 'r') as f:
            filelines = f.readlines()
        fileArray = np.zeros((1,3))
        iline = 0
        while iline < len(filelines):
            if filelines[iline].startswith('[') and filelines[iline].endswith(']\n'):
                lineWithNewLineChar = strip_new_line_char(filelines[iline])
                lineElements = lineWithNewLineChar.split(",")
                temp = np.zeros((1,3))
                temp[:,2] = float(lineElements[2][1:-1])
                temp[:,1] = float(lineElements[1][8:-2])
                temp[:,0] = float(lineElements[0][8:-2])
                fileArray = np.append(fileArray,temp,axis=0)
                iline += 1 
    return fileArray[1:,:]


In [16]:
folder = 'auditory_categorization_prolific_online_data/preprocess_results/results_from_cluster/resultsProlificData/veridicalGaussParameters_finerOptimization/'
subject = 'e453'
sensoryStd = 0.37
subjectArrayProb = bestFitParametersProbabilistic(folder, subject, sensoryStd)

folder = 'auditory_categorization_prolific_online_data/preprocess_results/results_from_cluster/resultsProlificData/pbackLesionModel/veridicalGaussParameters_finerOptimization/'
sensoryStd = 0.37
subjectArraySignal = bestFitParametersSignal(folder, subject, sensoryStd)

folder = 'auditory_categorization_prolific_online_data/preprocess_results/results_from_cluster/resultsProlificData/ControlResults/randomChoice/'
subjectArrayRandom = bestFitParametersRandom(folder, subject)

print("Stats for Probabilistic versus Signal",pg.wilcoxon(subjectArrayProb[:,-1], subjectArraySignal[:,-1]))
print(scipy.stats.norm.isf(0.64/ 2))
print("BIC for Probabilistic and Signal models",
      np.median(subjectArrayProb[:,-1]), np.median(subjectArraySignal[:,-1]))

print("Stats for Probabilistic versus Random",pg.wilcoxon(subjectArrayProb[:,-1], subjectArrayRandom[:,-1]))
print(scipy.stats.norm.isf(3.955912e-18/ 2))


Stats for Probabilistic versus Signal            W-val alternative     p-val       RBC    CLES
Wilcoxon  2390.0   two-sided  0.643755  0.053465  0.5012
0.46769879911450823
BIC for Probabilistic and Signal models 310.74934626962727 312.29295017361096
Stats for Probabilistic versus Random           W-val alternative         p-val  RBC  CLES
Wilcoxon    0.0   two-sided  3.955912e-18 -1.0   0.0
8.680051056440025
